In [1]:
# Important! This notebook runs on Python 3

import pandas as pd

In [36]:
# Necessary world information

# Prices and Costs
# Prices of one beer at each level of the supply chain.
retail_price = 10
wholesale_price = 9
regional_warehouse_price = 8
factory_price = 7
field_price = 6
# Cost of holding one beer during one day on warehouse.
# Assumed to be the same for all levels
warehouse_price = 0.5
# Cost of backlog: non fulfilled orders
backlog_cost = 0.5

# Initial Inventories
retail_ininv = 100
wholesale_ininv = 100
regional_warehouse_ininv = 100
factory_ininv = 100

# Necessary customer demand and field supply trends
# Customer demand will be added a small random effect each time
# Field will stay non-random
#customer_demand = pd.read_csv(beer_customer_demand.csv)
#field_supply = pd.read_csv(beer_field_supply.csv)

In [39]:
class Agent:
    """
    Creates a Beer Supply Chain Agent ready to start interacting
    with other agents.
    input:
    * name (string) indicating the type of agent, can be one of four:
    {Retail,Wholesale,Regional_Warehouse,Factory} 
    * inventory (numeric) starting inventory at day 1
    output: an object of type Agent
    """
    def __init__(self,name,inventory):
        
        # I am letting different levels have different selling and buying prices
        # This could also include different warehousing/backlogs
        if name == "Retail":
            self.selling_price = retail_price
            self.buying_price = wholesale_price
        elif name == "Wholesale":
            self.selling_price = wholesale_price
            self.buying_price = regional_warehouse_price
        elif name == "Regional_Warehouse":
            self.selling_price = regional_warehouse_price
            self.buying_price = factory_price
        elif name == "Factory":
            self.selling_price = factory_price
            self.buying_price = field_price
        
        self.name = name
        self.inventory = inventory
        self.total_warehousing_costs = 0
        self.total_money = 0
        self.backlog = 0
    
    def pay_for_warehousing(self):
        # Pays for warehousing of inventory: must be done either
        # "first thing in the morning" or "last time in the night"
        self.total_money = self.total_money - \
                self.inventory * warehouse_price
    
    def receive_upstream(self,orders):
        # Receives orders from upstream agent first thing in the morning
        self.inventory = self.inventory + orders
        self.total_money = self.total_money - \
                orders * self.buying_price
        
    def give_downstream(self,orders):
        # Checks if he has availability to fulfill order,
        # fulfills as much as he can
        if self.inventory >= orders:
            self.total_money = self.total_money + \
                orders * self.selling_price
            self.inventory = self.inventory - orders
            return orders
        else:
            orders_that_could_be_fulfilled = self.inventory
            # Sells all its inventory
            self.total_money = self.total_money + \
                self.inventory * self.selling_price
            # If there were non fulfilled orders, those cause a penalty
            self.backlog = (orders - self.inventory) * backlog_cost
            self.inventory = 0
            return orders_that_could_be_fulfilled

In [62]:
retail_agent = Agent("Retail", retail_ininv)
wholesale_agent = Agent("Wholesale", wholesale_ininv)
regional_warehouse_agent = Agent("Regional_Warehouse", regional_warehouse_ininv)
factory_agent = Agent("Factory",factory_ininv)

In [57]:
# THIS CELL IS HARDCODED BUT TRENDS COME FROM A CSV
# AND DEMANDS IS WHAT THE AGENTS WANT TO LEARN

# One day of transactions
# Hardcoding these - need to create seasonality for them
customer_demand = 51
field_supply = 60

# Demands come from time t-1
retail_demand = customer_demand  # hardcoded
wholesale_demand = retail_demand + 25  # hardcoded
regional_warehouse_demand = wholesale_demand + 25  # hardcoded
factory_demand = regional_warehouse_demand - 25  # hardcoded

In [58]:
# Everyone pays for warehousing overnight
factory_agent.pay_for_warehousing
regional_warehouse_agent.pay_for_warehousing
wholesale_agent.pay_for_warehousing
retail_agent.pay_for_warehousing

# Orders are fulfilled first time in the morning
# Everyone gets their shippings at the same time

fulfilled_to_factory = field_supply
factory_agent.receive_upstream(fulfilled_to_factory)

fulfilled_to_regional_warehouse = factory_agent.give_downstream(regional_warehouse_demand)
regional_warehouse_agent.receive_upstream(fulfilled_to_regional_warehouse)

fulfilled_to_wholesale = regional_warehouse_agent.give_downstream(wholesale_demand)
wholesale_agent.receive_upstream(fulfilled_to_wholesale)

fulfilled_to_retail = wholesale_agent.give_downstream(retail_demand)
retail_agent.receive_upstream(fulfilled_to_retail)

fulfilled_to_customer = retail_agent.give_downstream(customer_demand)

In [71]:
print('Retail agent asked for ' + str(retail_demand) + ' and received ' + str(fulfilled_to_retail))
print('Wholesale agent asked for ' + str(wholesale_demand) + ' and received ' + str(fulfilled_to_wholesale))
print('Regional Warehouse agent asked for ' + str(regional_warehouse_demand) + ' and received ' + str(fulfilled_to_regional_warehouse))
print('Factory agent asked for ' + str(factory_demand) + ' and received ' + str(fulfilled_to_factory))

Retail agent asked for 51 and received 51
Wholesale agent asked for 76 and received 76
Regional Warehouse agent asked for 101 and received 101
Factory agent asked for 76 and received 60


In [70]:
factory_agent.inventory

59